#Age Detection 
The face age estimation algorithm mainly uses mean absolute error (MAE) and cumulative score (CS) as the standard to measure the accuracy of age estimation.

Age detection works on the same basis as any other computer vision task. Large amounts of training data (pictures) for different ages are trained into the system to build a model. As time goes on the new incoming images are compared against the model to get a prediction of age.
https://www.facelytics.io/

In [1]:
!pip install openvino

     |████████████████████████████████| 28.9 MB 1.4 MB/s 


In [2]:
from openvino.inference_engine import IENetwork 
from openvino.inference_engine import IECore
import warnings
from google.colab.patches import cv2_imshow
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
def load_IR_to_IE(model_xml):
    ### Load the Inference Engine API
    plugin = IECore()
    ### Loading the IR files to IENetwork class
    model_bin = model_xml[:-3]+"bin" 
    network = IENetwork(model=model_xml, weights=model_bin)
    ### Loading the network
    executable_net = plugin.load_network(network,"CPU")
    print("Network succesfully loaded into the Inference Engine")
    return executable_net

In [4]:
def synchronous_inference(executable_net, image):
    ### Get the input blob for the inference request
    input_blob = next(iter(executable_net.inputs))
    ### Perform Synchronous Inference
    result = executable_net.infer(inputs = {input_blob: image})
    return result

In [5]:
#age detection age-gender-recognition-retail-0013
en = load_IR_to_IE('age.xml')
import cv2

imageName = "ageFive.png"
image = cv2.imread(imageName)
#face_img = cv2.dnn.blobFromImage(image, 1./127.5, (128, 128), (1, 1, 1), True)
resized = cv2.resize(image, (62,62), interpolation = cv2.INTER_AREA)

from torchvision import transforms
tran = transforms.ToTensor()  # Convert the numpy array or PIL.Image read image to (C, H, W) Tensor format and /255 normalize to [0, 1.0]
img_tensor = tran(resized)
##
img_tensor = img_tensor.unsqueeze_(0)
##
#print(img_tensor.size())  # (C,H, W), channel order (B, G, R)

res = synchronous_inference(en, img_tensor)
cv2_imshow(cv2.imread(imageName))
print("The predicted age is: "+str(round((86.5 - res['age_conv3'][0][0][0][0]*100/25 * 100) * 100)))

Exception: ignored

Sources:

https://towardsdatascience.com/age-detection-using-facial-images-traditional-machine-learning-vs-deep-learning-2437b2feeab2

https://github.com/jojo96/intel-openvino-colab/tree/main/notebooks

https://colab.research.google.com/drive/1YK7vAmOAznTPYy8G1I3NDof2x16ajY4M#scrollTo=Cl17ioro_6E6

https://www.facelytics.io/

https://github.com/alihussainia/openvino-colab

Useful models: https://github.com/jojo96/intel-openvino-colab/tree/main/models
